# Lab 2

In this lab, we will explore how caching and access order can affect the number
of memory accesses for tensor programs.

In [17]:
from loaders import *
from copy import deepcopy
import random
import numpy as np
from cache import Cache, CacheAssoc
answer(
    question='1.0',
    subquestion=f'What is your name?',
    answer= 'Melvin He',
    required_type=str,
)
answer(
    question='1.0',
    subquestion=f'What is your email address?',
    answer= 'melvin_he@brown.edu',
    required_type=str,
)
answer(
    question='1.0',
    subquestion=f'What is your Banner ID?',
    answer= 'B01717637',
    required_type=str,
)

1.0: What is your name?
	Melvin He
1.0: What is your email address?
	melvin_he@brown.edu
1.0: What is your Banner ID?
	B01717637


## Part 1: Cache Basics

## Matrix Multiplication and Memory Access Pattern

We will check the memory access pattern for matrix multiplication. Consider this matrix multiplication:

$$
C_{m,n} = \sum_k A_{m,k} \times B_{k,n}
$$

Suppose our processor doesn't have a cache and directly loads operands from the main memory and stores the results back to the main memory. We want to count how many memory accesses are necessary to complete the computation, and estimate how much energy will be consumed for the memory accesses. 

In [18]:
# Initialize values
seed = 10
random.seed(seed)

M = 4
K = 4
N = 4

# Matrix A (M x K)
a_MK = []
for m in range(M):
    a_MK.append([random.randint(1, 9) for i in range(K)])


# Matrix B (K x N)
b_KN = []
for k in range(K):
    b_KN.append([random.randint(1, 9) for i in range(N)])
    
print(a_MK)
print(b_KN)

# ground truth for the result (M x N)
c_MN = []
for m in range(M):
    temp = []
    for n in range(N):
        t = 0
        for k in range(K):
            t += a_MK[m][k] * b_KN[k][n]
        temp.append(t)
    c_MN.append(temp)
print(c_MN)

[[1, 7, 8, 1], [4, 8, 8, 5], [3, 1, 9, 8], [6, 2, 4, 6]]
[[1, 7, 3, 6], [7, 7, 5, 5], [8, 3, 5, 6], [3, 8, 4, 8]]
[[117, 88, 82, 97], [139, 148, 112, 152], [106, 119, 91, 141], [70, 116, 72, 118]]


### Question 1

How many memory reads are in the following loop nest? Break down the number of memory reads for each matrix (A, B, and C).

How many memory writes are in the following loop nest? Assume there is no register for storing partial sums, and reading and writing intermediate results should also be considered. Break down the number of memory writes for each matrix (A, B, and C).

In [19]:
# Loop nest for the matrix multiplication
c = np.zeros((M, N))
for m in range(M):
    for n in range(N):
        for k in range(K):
            # load a
            a = a_MK[m][k]

            # load b
            b = b_KN[k][n]

            # compute partial sum
            c[m][n] += a * b

# Check the result
print(np.all(c==np.asarray(c_MN)))

True


In [20]:
answer(
    question='1.1',
    subquestion=f'How many reads of A?',
    answer= 64,
    required_type=int,
)
answer(
    question='1.1',
    subquestion=f'How many reads of B?',
    answer= 64,
    required_type=int,
)
answer(
    question='1.1',
    subquestion=f'How many reads of C?',
    answer= 64,
    required_type=int,
)
answer(
    question='1.1',
    subquestion=f'How many writes of A?',
    answer= 0,
    required_type=int,
)
answer(
    question='1.1',
    subquestion=f'How many writes of B?',
    answer= 0,
    required_type=int,
)
answer(
    question='1.1',
    subquestion=f'How many writes of C?',
    answer= 64,
    required_type=int,
)

1.1: How many reads of A?
	64
1.1: How many reads of B?
	64
1.1: How many reads of C?
	64
1.1: How many writes of A?
	0
1.1: How many writes of B?
	0
1.1: How many writes of C?
	64


## Direct Mapped Cache

Suppose our processor uses a directed mapped cache. We want to count the number of cache hit and miss for both read and write operations. Assume all data within matrices are initially stored in the main memory in __row major__ order.  Matrices are stored in memory in alphabetical order relative to other matrices, with the first element of the A matrix having the lowest address.

We are using two parameters when defining a cache: `log_size` and `words_per_line`. The total number of __words__ that a cache can store is $2^{log\_size}$, and the number of words per line (block size) is `words_per_line`. For example, if a cache has `log_size=4`, and `words_per_line=2`, this cache can store total 16 words with block size of 2. We assume that our direct mapped cache uses write-through with no write allocation for store operations. 

In [21]:
def read_addr(addr, memory, cache):
    val = memory[addr]
    cache.load(addr)
    return val
    
def write_addr(addr, memory, cache, val=0):
    memory[addr] = val
    cache.store(addr)

### Question 2

Modify the address that the processor has to request from memory for the read and write operations in the loop nest from Question 1.1.

In [26]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_b
            addr_b = M * K + k * N + n
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_c
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))

# Check the result
answer(  # DO NOT MODIFY
    question='1.2',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)


# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

1.2: Check if result is correct.
	True
-------Cache A--------
Cache Statistics:
cache rd: 48
cache wr: 0
mem rd: 16
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 48
cache wr: 64
mem rd: 16
mem wr: 64


### Question 3

Now, run the same loop nest using caches with the same size, but with different words per line (block size). Explain why the statistics for Cache A is different from the previous case. The modifications to the caches have already been provided for you.

In [27]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=2)
cache_b = Cache(log_size=3, words_per_line=2)
cache_c = Cache(log_size=3, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_b
            addr_b = M * K + k * N + n
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_c
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))

# Check the result
answer(  # DO NOT MODIFY
    question='1.3',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

1.3: Check if result is correct.
	True
-------Cache A--------
Cache Statistics:
cache rd: 56
cache wr: 0
mem rd: 8
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 56
cache wr: 64
mem rd: 8
mem wr: 64


In [28]:
answer(
    question='1.3',
    subquestion="The larger block size exploits _____ locality.",
    answer= 'spatial',
    required_type=('spatial', 'temporal')
)

1.3: The larger block size exploits _____ locality.
	spatial


### Question 4

Suppose we store data of Matrix B (and only Matrix B) to the main memory in __column major__ order. Change the address that are requested by the processor, and explain the difference of the statistics for Cache B.

In [29]:
mem_log_size = 10
memory_col = np.zeros(2**mem_log_size).astype(np.int32)
memory_col[:M*K] = np.asarray(a_MK).flatten()
memory_col[M*K:M*K+K*N] = np.asarray(b_KN).transpose().flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=2)
cache_b = Cache(log_size=3, words_per_line=2)
cache_c = Cache(log_size=3, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory_col, cache_a)
            
            # Your code: modify addr_b
            addr_b = M * K + n * K + k
            b = read_addr(addr_b, memory_col, cache_b)
            
            # Your code: modify addr_c
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory_col, cache_c)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory_col, cache_c, int(psum))

# Check the result
answer(  # DO NOT MODIFY
    question='1.4',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

1.4: Check if result is correct.
	True
-------Cache A--------
Cache Statistics:
cache rd: 56
cache wr: 0
mem rd: 8
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 32
cache wr: 0
mem rd: 32
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 56
cache wr: 64
mem rd: 8
mem wr: 64


In [30]:
answer(
    question='1.4',
    subquestion="Arranging B in column-major order creates _____ locality in the access pattern of B.",
    answer= 'spatial',
    required_type=('spatial', 'temporal')
)

answer(
    question='1.4',
    subquestion="The traversal pattern due to the M, N, K loop nest is ____ with the row-major format of B.",
    answer= 'discordant',
    required_type=('discordant', 'concordant')
)

answer(
    question='1.4',
    subquestion="The traversal pattern due to the M, N, K loop nest is ____ with the column-major format of B. Answer with 'discordant' or 'concordant'",
    answer= 'concordant',
    required_type=('discordant', 'concordant')
)

1.4: Arranging B in column-major order creates _____ locality in the access pattern of B.
	spatial
1.4: The traversal pattern due to the M, N, K loop nest is ____ with the row-major format of B.
	discordant
1.4: The traversal pattern due to the M, N, K loop nest is ____ with the column-major format of B. Answer with 'discordant' or 'concordant'
	concordant


### Question 5

Suppose our loop nest order is different, from M -> N -> K to K -> N -> M.
Assume all conditions are identical to those in Question 2. Explain cache
statistics compared to those in Question 2.


For this lab, consider the following three types of cache misses.

| Miss type | Description |
| --------- | ----------- |
| Compulsory miss | The first access to any data will be a miss. |
| Capacity miss   | When the accessed data has already been fetched but also evicted due to insufficient cache capacity. |
| Conflict miss   | When the accessed data has already been fetched but also evicted due to another data mapped to the same cache line. |

In [31]:
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

for k in range(K):
    for n in range(N):
        for m in range(M):
            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory, cache_a)
            
            # Your code: modify addr_a 
            addr_b = M * K + k * N + n
            b = read_addr(addr_b, memory, cache_b)
            
            # Your code: modify addr_a 
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory, cache_c)

            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache_c, int(psum))


# Check the result
answer(  # DO NOT MODIFY
    question='1.5',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

1.5: Check if result is correct.
	True
-------Cache A--------
Cache Statistics:
cache rd: 0
cache wr: 0
mem rd: 64
mem wr: 0
-------Cache B--------
Cache Statistics:
cache rd: 48
cache wr: 0
mem rd: 16
mem wr: 0
-------Cache C--------
Cache Statistics:
cache rd: 0
cache wr: 64
mem rd: 64
mem wr: 64


In [32]:
answer(
    question='1.5',
    subquestion="Does the cache for tensor A have (1) compulsory, (2) capacity, and (3) conflict misses? Please write your answer (True/False in Python) in order as a list (e.g., [True, False, True]).",
    answer= [True, False, False], # Answer here
    required_type=[bool, bool, bool],
)
answer(
    question='1.5',
    subquestion="Does the cache for tensor B have (1) compulsory, (2) capacity, and (3) conflict misses? Please write your answer (True/False in Python) in order as a list (e.g., [True, False, True]).",
    answer= [True, False, True], # Answer here
    required_type=[bool, bool, bool],
)
answer(
    question='1.5',
    subquestion="Does the cache for tensor C have conflict misses? Please write your answer (True/False in Python).",
    answer= True,
    required_type=bool,
)
answer(
    question='1.5',
    subquestion="Assume for this subquestion that address conflicts can be completely avoided by adding some mechanism in the cache for tensor C, would there still be (1) compulsory and (2) capacity misses? Please write your answer (True/False in Python) in order as a list (e.g., [False, True]).",
    answer= [True, False], # Answer here
    required_type=[bool, bool],
)

1.5: Does the cache for tensor A have (1) compulsory, (2) capacity, and (3) conflict misses? Please write your answer (True/False in Python) in order as a list (e.g., [True, False, True]).
	[True, False, False]
1.5: Does the cache for tensor B have (1) compulsory, (2) capacity, and (3) conflict misses? Please write your answer (True/False in Python) in order as a list (e.g., [True, False, True]).
	[True, False, True]
1.5: Does the cache for tensor C have conflict misses? Please write your answer (True/False in Python).
	True
1.5: Assume for this subquestion that address conflicts can be completely avoided by adding some mechanism in the cache for tensor C, would there still be (1) compulsory and (2) capacity misses? Please write your answer (True/False in Python) in order as a list (e.g., [False, True]).
	[True, False]


## Set Associative Cache

In previous questions, we considered using one cahce per tensor. Now, we will see what happens if we use only one cache for all tensors (Question 6). You should see that the number of misses increase.

Then, we will use a set-associative cache to reduce misses (Question 7). This cache has another parameter, `num_ways`, that specifies the number of ways. For example, if `log_size=4`, `words_per_line=2`, and `num_ways=2`, this cache can store total 16 words and there are total 4 sets, where each set has two ways and the block size will be two. Similar to the direct mapped cache we discussed above, our set associative cache uses write-through with no write allocation for store, and uses Bit Pseudo Least Recently Used to replace cache entries.

### Question 6

Let's run the same loop nest as in Question 2, except for that we use one large cache instead of using three different caches for each matrix. Note that we are still using a direct mapped cache here.

In [33]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache = Cache(log_size=5, words_per_line=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory, cache)
            
            # Your code: modify addr_b
            addr_b = M * K + k * N + n
            b = read_addr(addr_b, memory, cache)
            
            # Your code: modify addr_c
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory, cache)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache, int(psum))
            
            # debug
            # print("---------------------")
            # print(addr_a, addr_b, addr_c)

# Check the result
answer(  # DO NOT MODIFY
    question='1.6',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)

# Print cache statistics
print("-------Cache--------")
cache.print_stats()

1.6: Check if result is correct.
	True
-------Cache--------
Cache Statistics:
cache rd: 108
cache wr: 64
mem rd: 84
mem wr: 64


In [34]:
answer(
    question='1.6',
    subquestion="Compared to using three separate caches, the number of misses has increased. This increase is due to more ___ misses.",
    answer= 'conflict',
    required_type=('compulsory', 'conflict', 'capacity')
)

1.6: Compared to using three separate caches, the number of misses has increased. This increase is due to more ___ misses.
	conflict


### Question 7

Observe the cache access pattern when we now use a set associative cache with 2 ways. Explain the difference compared to Question 6.

In [36]:
# Define a main memory
mem_log_size = 10
memory = np.zeros(2**mem_log_size).astype(np.int32)
memory[:M*K] = np.asarray(a_MK).flatten()
memory[M*K:M*K+K*N] = np.asarray(b_KN).flatten()

# Define a cache for each matrix (A, B, C)
cache = CacheAssoc(log_size=5, words_per_line=2, num_ways=2)

# Loop nest for matrix multiplication
for m in range(M):
    for n in range(N):
        for k in range(K):

            # Your code: modify addr_a 
            addr_a = m * K + k
            a = read_addr(addr_a, memory, cache)
            
            # Your code: modify addr_b
            addr_b = M * K + k * N + n
            b = read_addr(addr_b, memory, cache)
            
            # Your code: modify addr_c
            addr_c = M * K + K * N + m * N + n
            psum = read_addr(addr_c, memory, cache)
            
            # compute partial sum
            psum += a * b
            write_addr(addr_c, memory, cache, int(psum))
            
            # debug
            # print("---------------------")
            # print(addr_a, addr_b, addr_c)

# Check the result
answer(  # DO NOT MODIFY
    question='1.7',
    subquestion=f'Check if result is correct.',
    answer=bool(np.all(memory[M*K+K*N:M*K+K*N+M*N].reshape((M, N))==np.asarray(c_MN))),
    required_type=bool,
)

# Print cache statistics
print("-------Cache--------")
cache.print_stats()

1.7: Check if result is correct.
	True
-------Cache--------
Cache Statistics:
cache rd: 126
cache wr: 64
mem rd: 66
mem wr: 64


In [37]:
answer(
    question='1.7',
    subquestion="Compared to using one cache, the set-associative cache reduces misses.",
    answer= True,
    required_type=bool,
)

answer(
    question='1.7',
    subquestion="Set-associative caches are beneficial for reducing _____ misses.",
    answer= 'conflict',
    required_type=('compulsory', 'conflict', 'capacity'),
)

1.7: Compared to using one cache, the set-associative cache reduces misses.
	True
1.7: Set-associative caches are beneficial for reducing _____ misses.
	conflict


## Important Takeaways

Set-associative caches can reduce certain types of misses compared to direct-mapped caches. However, as you have seen, those misses can also be avoided by co-designing the data layout (e.g., row-major or column-major), the dataflow (the loop order), and the cache design. Finally, note that set-associative caches come with extra overhead since tag matches must be performed for all associativity.